In [3]:
import casperfpga
import lwa_f
import pandas as pd
import time

## Work on extracting and writing to specific bits of a number

In [2]:
### test: say I have an 8 bit number n and I want to write a 3 bit number r to MSB-2:MSB-5
nbits=8
n = 245
b = (bin(n))
s = str(b) 
print(b)
print(b[1])

MSBoffset=2
bw=3
LSBoffset=nbits-(MSBoffset+bw)

0b11110101
b


In [3]:
#extract the old r
r_old = (n&0b00111000)>> LSBoffset
print(r_old)
print(bin(r_old))

6
0b110


In [4]:
# set a new r
r_new = 5
print(bin(r_new))
print(bin(r_new<<LSBoffset))

#remove old r and add new
n_new=(n&0b11000111) + (r_new<<LSBoffset)
print(n_new, bin(n_new))

0b101
0b101000
237 0b11101101


In [5]:
# now do the above again, but calculate the extract mask (0b00111000 in the example) and the exclude mask (0b11000111)
extractmask = ((1<<bw) - 1) <<LSBoffset
print(bin(extractmask))
excludemask = ((1<<nbits)-1) - extractmask
print(bin(excludemask))

0b111000
0b11000111


In [4]:
def extractvalue(mainregister,nbits,MSBoffset,bw):
    #extract the value of the specified consecutive bits of mainregister
    #MSBoffset is the offset from the most significant bit of mainregister
    #bw is the bitwidth of the number to extract
    #nbits is the number of bits of mainregister
    #returns the extracted value
    LSBoffset=nbits-(MSBoffset+bw)
    extractmask = ((1<<bw) - 1) <<LSBoffset
    val=(mainregister&extractmask)>> LSBoffset
    return val

def updatevalue(mainregister,nbits,MSBoffset,bw,newval):
    #update the value of the specified consecutive bits of mainregister
    #MSBoffset is the offset from the most significant bit of mainregister
    #bw is the bitwidth of the number to extract
    #nbits is the number of bits of mainregister
    #newval is the new value to write to that subset of bits
    #returns updated value of mainregister
    LSBoffset=nbits-(MSBoffset+bw)
    extractmask = ((1<<bw) - 1) <<LSBoffset
    excludemask = ((1<<nbits)-1) - extractmask
    mainregister=(mainregister&excludemask) + (newval<<LSBoffset)
    return mainregister

In [53]:
print(extractvalue(245,32,26,3))
print(updatevalue(245,32,26,3,5))

6
237


## Lookup information about a setting (e.g. what bits in what register is it) from a spreadsheet


In [4]:
registers = pd.read_excel("cr_registers.xlsx")
print(registers.columns)

Index(['interface', 'bitwidth', 'registername', 'offset_from_msb',
       'mainregister_bitwidth', 'readonly', 'description'],
      dtype='object')


In [5]:
registers

,interface,bitwidth,registername,offset_from_msb,mainregister_bitwidth,readonly,description
0,enable_coinc_trig,1,cosmic_ray_triggercontrol,0,32,readwrite,choose whether to allow or ignore internally-g...
1,reset_to_listen,1,cosmic_ray_triggercontrol,1,32,readwrite,put the board into listening state
2,delay_trigger,12,cosmic_ray_triggercontrol,2,32,readwrite,"delay the internally-generated trigger, in cas..."
3,wait_after_readout,12,cosmic_ray_triggercontrol,14,32,readwrite,number of clock cycles to ignore new INTERNAL ...
4,send_trigger,1,cosmic_ray_triggercontrol,26,32,readwrite,send a trigger from software
5,trig_debug_reset,1,cosmic_ray_triggercontrol,27,32,readwrite,reset trigger counter
6,trig_debug_timer_reset,1,cosmic_ray_triggercontrol,28,32,readwrite,reset trigger loop test timer
7,snapshot_count_reset,1,cosmic_ray_triggercontrol,29,32,readwrite,reset snapshot counter
8,select_input_signal,2,cosmic_ray_triggercontrol,30,32,readwrite,select input data source
9,dest_ip,32,cosmic_ray_cr_dest_ip,0,32,readwrite,destination IP address for cosmic ray data


In [6]:
interface = registers[registers['interface']=='enable_coinc_trig']

In [7]:
print(interface['description'][0])

choose whether to allow or ignore internally-generated registers


In [5]:
def lookup_register(name,fname):
    #fname is the file with a table of register information
    #name is the name you want to look up
    registers = pd.read_excel(fname)
    return registers[registers['interface']==name]

def getvalue(brd,name,fname):
    #brd is a casperfpga CasperFpga object
    #fname is the file name of the register description table
    #name is the name of the quantity you want to read from the board
    info=lookup_register(name,fname)
    mainregistername=info['registername'].values[0]
    mainregistervalue=brd.read_int(mainregistername)
    value=extractvalue(mainregistervalue,info['mainregister_bitwidth'].values[0],info['offset_from_msb'].values[0],info['bitwidth'].values[0])
    time.sleep(0.01)
    return value

def setvalue(brd,name,fname,newvalue):
    #brd is a casperfpga CasperFpga object
    #fname is the file name of the register description table
    #name is the name of the setting you want to update on the board
    #newvalue is the new value you want to update that setting with
    info=lookup_register(name,fname)
    mainregistername=info['registername'].values[0]
    mainregistervalue=brd.read_int(mainregistername)
    #value=extractvalue(mainregistervalue,info['mainregister_bitwidth'].values[0],info['offset_from_msb'].values[0],info['bitwidth'].values[0])
    updatedvalue=updatevalue(mainregistervalue,info['mainregister_bitwidth'].values[0],info['offset_from_msb'].values[0],info['bitwidth'].values[0],newvalue)
    brd.write_int(mainregistername,updatedvalue)
    time.sleep(0.01)
    return


In [9]:
lookup_register('almostfull','cr_registers.xlsx')

,interface,bitwidth,registername,offset_from_msb,mainregister_bitwidth,readonly,description
27,almostfull,32,cosmic_ray_forty_g_tx_afull,0,32,readonly,Counts number of times the almost full port on...


In [10]:
information = lookup_register('almostfull','cr_registers.xlsx')
print(information['registername'].values[0])

cosmic_ray_forty_g_tx_afull


## Use the previous work to re-write the SNAP2 control functions

In [2]:
f=casperfpga.CasperFpga("snap03")

In [44]:
f.upload_to_ram_and_program("../../snap2_f_200msps_64i_4096c/outputs/snap2_f_200msps_64i_4096c_2021-09-02_2135TIMINGFAIL.fpg")

100% |########################################################################|


In [55]:
setvalue(f,'cr_dest_port','cr_registers.xlsx',11111)
print(getvalue(f,'cr_dest_port','cr_registers.xlsx'))


91021312
cosmic_ray_eth_control
11111


In [73]:
#read all the registers
for v in registers['interface'].values:
    print(v, getvalue(f,v,'cr_registers.xlsx'))

enable_coinc_trig 0
reset_to_listen 1
delay_trigger 0
wait_after_readout 0
send_trigger 0
trig_debug_reset 0
trig_debug_timer_reset 0
snapshot_count_reset 0
select_input_signal 0
dest_ip 170459242
trigger_power_thresh 0
veto_power_thresh 0
trigger_window 0
veto_window 0
antenna_number_thresh 0
veto_number_thresh 0
trigger_antennas1 0
trigger_antennas2 0
veto_antennas1 0
veto_antennas2 0
wait_between_packets 100
eth_enable 0
readout_timer_reset 0
cr_rst_40g 0
cr_dest_port 11111
forty_g_up 1
forty_g_tx 0
almostfull 2
overflow 0
eof 0
eof_and_valid 0
readout_state_timer 0
count_triggers 2
trig_debug_timer 581777968
count_snapshots 1
readout_state_value 0


In [20]:
#functions needed for readout.py
def reset_to_listen(brd):
    setvalue(brd,'reset_to_listen','cr_registers.xlsx',0)
    setvalue(brd,'reset_to_listen','cr_registers.xlsx',1)
    return

def setup_ethernet(brdname,brd,fpgfile,destinationcomputer,packetwait):
    #this function configures the 40Gb block and sets the destination IP and Port and the number of clock cycles between packets
    #It does not enable the valid and end of frame signals, so no data will be sent until running enable_ethernet(brd)
    #brdname is the network name of a SNAP2
    #brd is a casperfpga object that connects to that SNAP2
    #fpgfile is the path to the fpgfile that is already programmed on the board
    #destinationcomputer is the name, either "minor" or "lwacr", of the computer to which to send the data
    #packetwait is the number of clock cycles to wait between packets
    ######################## Define Addresses #############################
    if brdname=='snap2-rev2-9':
        ip = '192.168.41.13'
        mac = 0x020202030303
    if brdname=='snap2-rev2-11':
        ip='192.168.41.14'
        mac=0x020202040404
    if brdname=='snap2-rev2-12':
        ip='192.168.41.15'
        mac=0x020202050505
    if brdname=='snap01':
        ip='10.41.0.201'
        mac=0x020202010101
    if brdname=='snap03':
        ip='10.41.0.203'
        mac=0x020202030303
    #configure the 40 Gbe core
    brd.get_system_information(fpgfile)
    if destinationcomputer == 'lwacr':
        setvalue(brd,'dest_ip','cr_registers.xlsx',(10<<24)+(41<<16)+(0<<8)+106)
        setvalue(brd,'cr_dest_port','cr_registers.xlsx',11111)
        brd.gbes.cosmic_ray_cr_forty_gbe.configure_core(mac, ip, 11111)
        brd.gbes.cosmic_ray_cr_forty_gbe.set_single_arp_entry('10.41.0.106',  0x043f72dfc2f8)
        brd.gbes.cosmic_ray_cr_forty_gbe.print_gbe_core_details(arp=True)

    elif destinationcomputer == 'minor':
        setvalue(brd,'dest_ip','cr_registers.xlsx',3232246028)
        setvalue(brd,'cr_dest_port','cr_registers.xlsx',11111)
        brd.gbes.cosmic_ray_cr_forty_gbe.configure_core(mac, ip, 11111)
        brd.gbes.cosmic_ray_cr_forty_gbe.set_single_arp_entry('192.168.41.12', 0x98039b3d8b7b)
        brd.gbes.cosmic_ray_cr_forty_gbe.print_gbe_core_details(arp=True)
    else:
        print(destination + " is not a recognized destination.")
    
    #set wait time
    if packetwait:
        print("Setting wait between packets to "+str(packetwait))
        setvalue(brd,'wait_between_packets', 'cr_registers.xlsx', packetwait)

    #reset the board to listening
    reset_to_listen(brd)
    
    return
    
def setup_data_source(brd,datasource):
    #set the data input to the cosmic ray system
    #brd is a casperfgpa.CasperFpga object
    #source is the data to use, either "constant", "counter", or "adc"
    #if constant is chosen, each timeseries will be a constant value, with a different value in place of each antenna signa
    #if counter is chosen, all timeseries will be the same ramp in time
    #if adc is chosen, data output from the ADC block will be used. This can be a sky signal or a test vector depending on how the ADC block is initialized
    
    if datasource=='constant':
        setvalue(brd,'select_input_signal', 'cr_registers.xlsx', 1)
    elif datasource=='counter':
        setvalue(brd,'select_input_signal', 'cr_registers.xlsx', 0)

    elif datasource=='adc':
        setvalue(brd,'select_input_signal', 'cr_registers.xlsx', 3)
    else:
        print("datasource must be 'constant','counter', or 'adc'. Defaulting to 'counter'")
    print("Data source set to "+datasource)
    return

def software_trigger(brd,stats):
    #Send a trigger from software
    #brd is a casperfgpa.CasperFpga object
    #if stats has a Boolean value of true, some diagnostic information will be printed

    #Ensure the board is listening: write the register reset_to_listen to 0 and then to 1. Confirm that it is reset.
    reset_to_listen(brd)
    val=getvalue(brd,"readout_state_value","cr_registers.xlsx")
    assert val==0, "board didn't reset to listen"

    if stats:
        #Reset the readout timer by putting readout_timer_reset1 to 0 and then to 1, and make sure the timer is staying zero and not counting.
        setvalue(brd,'readout_timer_reset','cr_registers.xlsx',0)
        setvalue(brd,'readout_timer_reset','cr_registers.xlsx',1)
        val=getvalue(brd,'readout_state_timer','cr_registers.xlsx')
        assert val == 0, "readout state timer started before trigger"

        #Read tx_count_eof_and_valid and overflow counters before sending the trigger. This counts how many packets the board thinks are sent and whether the packetizer buffer ever overflowed or almost overflowed..
        previous_packets = getvalue(brd,'eof_and_valid','cr_registers.xlsx')
        previous_afull = getvalue(brd,'almostfull','cr_registers.xlsx')
        previous_overflows = getvalue(brd,'overflow','cr_registers.xlsx')
    
    #Read the readout state register: Read readout_state_value. It should be 0.
    val =getvalue(brd,'readout_state_value','cr_registers.xlsx')
    assert val == 0, "board in readout state before trigger sent"

    #Send a trigger from software: write register send_trigger to 0 then to 1.
    setvalue(brd,"send_trigger","cr_registers.xlsx",0)
    setvalue(brd,"send_trigger","cr_registers.xlsx",1)

    if stats:
        #wait a moment
        time.sleep(0.1)
        ##############################Read registers after the readout ###############
        #After short wait, read tx_count_eof_and_valid, and confirm that it has incremented by 64.
        packets = getvalue(brd,'eof_and_valid','cr_registers.xlsx')
        print ("Packetizer formed " + str(packets - previous_packets) + " packets")
    
        #Read readout_state_value. It should be 0.
        val = getvalue(brd,'readout_state_value','cr_registers.xlsx')
        if val == 0:
            print("Board successfully returned to listening state after readout")
        if val !=0:
            print("Board did not return to listening state after readout. State value = " + str(val))
    
        #Read register readout_state_timer.
        val = getvalue(brd,'readout_state_timer','cr_registers.xlsx')
        print ("Readout took " + str(val) + " clock cycles")
    
        #Read overflow counter to confirm that the 40Gbe block did not overflow.
        of = getvalue(brd,'overflow','cr_registers.xlsx')
        overflows = of - previous_overflows
        if overflows == 0:
            print("Packetizer never overflowed")
        else:
            print("Packetizer overflowed" + str(val) + " times")
        afull = getvalue(brd,'almostfull','cr_registers.xlsx')
        new_afull = afull - previous_afull
        print("Ethernet block almost-full counter incremented by " + str(new_afull))
        return


In [19]:
#skeleton of readout.py script
#placeholder for argparse stuff, not included in notebook
brdname='snap03'
fpgfile='../../snap2_f_200msps_64i_4096c/outputs/snap2_f_200msps_64i_4096c_2021-09-02_2135TIMINGFAIL.fpg'
destinationcomputer='lwacr'
packetwait=100
Ntriggers = 3
triggerwait = 10
#load firmware --use lwa_f now, and initialize etc
#TODO add this section
brd = casperfpga.CasperFpga(brdname, transport=casperfpga.TapcpTransport)

#configure ethernet
setup_ethernet(brdname,brd,fpgfile,destinationcomputer,packetwait)

#configure data source
setup_data_source(brd,'counter')

#enable ethernet
setvalue(brd,'eth_enable','cr_registers.xlsx',1)

#send however many software triggers
for i in range(Ntriggers):
    software_trigger(brd,1)
    time.sleep(triggerwait)


------------------------
snap03:cosmic_ray_cr_forty_gbe configuration:
MAC:  02:02:02:03:03:03
Gateway:  0.0.0.1
IP:  10.41.0.203
Fabric port: 
11111
Disabled
	base_ip: 255.255.255.255
	ip_mask: 255.255.255.255
	rx_ips: []
ARP Table: 
IP: 10.41.0.106: MAC: 04:3F:72:DF:C2:F8
Setting wait between packets to 100
Data source set to counter
Packetizer formed 64 packets
Board successfully returned to listening state after readout
Readout took 22855 clock cycles
Packetizer never overflowed
Ethernet block almost-full counter incremented by 0
Packetizer formed 64 packets
Board successfully returned to listening state after readout
Readout took 22848 clock cycles
Packetizer never overflowed
Ethernet block almost-full counter incremented by 0
Packetizer formed 64 packets
Board successfully returned to listening state after readout
Readout took 22848 clock cycles
Packetizer never overflowed
Ethernet block almost-full counter incremented by 0


In [15]:
#additional functions needed to start observing with coincidencer

In [16]:
#start observing script

In [ ]:
#draft threshold scan script